<a href="https://colab.research.google.com/github/mostly-sunny/digital-health-hackathon/blob/main/3.%20coxph_find_best_network_by_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pycox - CoxPH Model
- Network : Test with sets, and find best network and lr by lowest brier score
- Input Variables : G1 ~ G300, Var1 ~ Var10, Treatment
- Output Variables : time, event
- Scaler : MinMaxScaler -> Var1 ~ Var10



In [1]:
pip install pycox

     |████████████████████████████████| 73 kB 1.3 MB/s 
     |████████████████████████████████| 65 kB 3.5 MB/s 
     |████████████████████████████████| 41 kB 699 kB/s 
     |████████████████████████████████| 1.9 MB 18.0 MB/s 
     |████████████████████████████████| 126 kB 53.2 MB/s 
     |████████████████████████████████| 2.2 MB 29.6 MB/s 
     |████████████████████████████████| 357 kB 39.2 MB/s 


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn_pandas import DataFrameMapper
import pandas as pd

import torch
import torchvision
import torchtuples as tt

from pycox.models import CoxPH
from pycox.evaluation import EvalSurv

In [4]:
torch.cuda.is_available()

True

In [5]:
np.random.seed(123456)
_ = torch.manual_seed(123456)

- all-in-one.csv 파일은 유전자 변이 유무, 임상 변수, 생존 기간, 사망 여부, 치료 유무가 열로 존재하는 파일
- test-data-treat-and-untreat.csv 파일은 all-in-one의 열은 같은 602개의 데이터.
  - (0번째 행) : 유전자 변이 모두 0, 치료 0
  - (1번째 행) : 유전자 변이 모두 0, 치료 1
  - (2~301번째 행) : 유전자 변이 n-1에만 1, 치료 0
  - (302~601번재 행) : 유전자 변이 n-301에만 1, 치료 1
- pandas 라이브러리에 있는 csv 파일을 DataFrame으로 바꾸어주는 read_csv 함수를 이용하여 파일을 읽어 들임.
- DataFrame은 표를 나타내는 데이터 타입임.

In [6]:
dataset = pd.read_csv('/content/all-in-one.csv')
dataset_for_hr = pd.read_csv('/content/test-data-treat-and-untreat.csv')

- 위에서 읽어들인 dataset 중에서 20%는 검증(_val -> validation)을 위해 sampling 한다.
- 남은 80%의 데이터에서도 20%는 테스트(_test)를 위해 sampling 한다.

In [7]:
dataset_val = dataset.sample(frac=0.2, random_state = np.random.seed(123456))
dataset_train = dataset.drop(dataset_val.index)
dataset_test = dataset_train.sample(frac=0.2, random_state = np.random.seed(123456))
dataset_train = dataset_train.drop(dataset_test.index)

In [8]:
dataset_val

,G1,G2,G3,G4,G5,G6,G7,G8,G9,G10,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20,G21,G22,G23,G24,G25,G26,G27,G28,G29,G30,G31,G32,G33,G34,G35,G36,G37,G38,G39,G40,...,G274,G275,G276,G277,G278,G279,G280,G281,G282,G283,G284,G285,G286,G287,G288,G289,G290,G291,G292,G293,G294,G295,G296,G297,G298,G299,G300,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,time,event,Treatment
601,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,5,4,6,4,3,3,5,7,1,71.955766,1,1
357,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,2,2,2,4,2,4,0,2,1,39.587305,1,0
945,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,3,4,1,5,6,2,1,2,2,3,99.459796,1,1
828,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,2,1,5,3,6,6,3,3,1,2,67.195341,1,1
649,1,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,3,1,2,4,0,1,1,0,6,35.031129,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,4,3,1,4,2,1,4,4,1,49.549464,1,0
642,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,5,7,4,8,7,2,4,3,1,1,43.761084,1,1
838,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,4,3,4,3,4,1,3,4,6,63.350415,0,1
570,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,3,2,4,2,3,6,1,4,2,2,42.715412,1,1


- columns_standardize : 임상변수 - 0~9사이의 값을 표준화
- columns_leave : 유전자 변이 유무 + 치료 유무 - 0과 1로 표현돼 있기 때문에 표준화 필요 없음.
- DataFrameMapper는 pandas DataFrame에서 원하는 열을 뽑아서 리스트로 만들어줌.
- 리스트로 만들때 StandardScaler()가 포함된 열은 표준화를 시킨 뒤, 그리고 None이면 갖고 있는 값을 그대로 넣음.

In [9]:
columns_standardize = ['Var' + str(i) for i in range(1,11)]
columns_leave = ['G' + str(i) for i in range(1,301)]
columns_leave += ['Treatment']

# standardize = [([col], StandardScaler()) for col in columns_standardize]
standardize = [([col], MinMaxScaler()) for col in columns_standardize]

leave = [(col, None) for col in columns_leave]

x_mapper = DataFrameMapper(leave + standardize)

- 위에서 만든 DataFrameMapper로 DataFrame 중 x(입력) 데이터를 모델이 학습할 수 있게끔 리스트 형식으로 바꾸어 준다.



In [10]:
x_train = x_mapper.fit_transform(dataset_train).astype('float32')
x_val = x_mapper.transform(dataset_val).astype('float32')
x_test = x_mapper.transform(dataset_test).astype('float32')
x_for_hr = x_mapper.transform(dataset_for_hr).astype('float32')

- DataFrame (표)에서 Y(출력)데이터인 time(생존기간)과 event(사망여부)를 뽑아 출력 데이터를 추린다.
- 검증(Validation)을 위한 입력-출력 세트 val을 만든다.

In [11]:
get_target = lambda df: (df['time'].values, df['event'].values)
y_train = get_target(dataset_train)
y_val = get_target(dataset_val)

durations_test, events_test = get_target(dataset_test)
val = x_val, y_val

함수 make_net : network을 생성해 리턴하는 함수
- input과 output의 노드 수, 은닉층 수, 은닉층의 노드 수 설정 가능

In [12]:
def make_net(in_features, out_features, hidden, nodes):
  if hidden == 1:
    network =  torch.nn.Sequential(
      torch.nn.Linear(in_features, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),
            
      torch.nn.Linear(nodes, out_features)
    )
  elif hidden == 2:
    network =  torch.nn.Sequential(
      torch.nn.Linear(in_features, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),

      torch.nn.Linear(nodes, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),
            
      torch.nn.Linear(nodes, out_features)
    )
  elif hidden == 3:
    network =  torch.nn.Sequential(
      torch.nn.Linear(in_features, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),

      torch.nn.Linear(nodes, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),

      torch.nn.Linear(nodes, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),
            
      torch.nn.Linear(nodes, out_features)
    )
  elif hidden == 4:
    network =  torch.nn.Sequential(
      torch.nn.Linear(in_features, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),

      torch.nn.Linear(nodes, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),

      torch.nn.Linear(nodes, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),
      
      torch.nn.Linear(nodes, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),

      torch.nn.Linear(nodes, out_features)
    )
  return network

- in_features : 입력데이터의 개수 (x_train.shape : 311 = 300(유전자) + 10(임상변수) + 1(치료유무))
- out_features : 출력노드의 개수

- hidden_layers : 은닉층 수를 가지고 있는 리스트
- number_nodes : 은닉층에 있는 노드 수를 가지고 있는 리스트
- learning_rates : 테스트할 학습률을 가지고 있는 리스트
- brier_scores = brier score을 계산해 append

In [13]:
in_features = x_train.shape[1]
out_features = 1

hidden_layers = [1,2]
number_nodes = [2000, 3000]
learning_rates = [0.0001, 0.001, 0]
brier_scores = []

total_num = len(hidden_layers) * len(number_nodes) * len(learning_rates)
count = 1

for i in hidden_layers:
  for j in number_nodes:
    for k in learning_rates:
      print(count, '/' , total_num)
      net = make_net(in_features, out_features, i, j)
      model = CoxPH(net, tt.optim.Adam)
      batch_size = 639

      if k == 0:
        lrfinder = model.lr_finder(x_train, y_train, batch_size, tolerance = 10)
        model.optimizer.set_lr(lrfinder.get_best_lr())
      else:
        model.optimizer.set_lr(k)
      
      epochs = 512
      callbacks = [tt.callbacks.EarlyStopping()]
      verbose = True

      %%time
      model.fit(x_train, y_train, batch_size, epochs, callbacks, verbose, val_data=val, val_batch_size=batch_size)
      _ = model.compute_baseline_hazards()
      surv = model.predict_surv_df(x_test)

      count += 1

      # calculate ratio
      log_partial_hazard = model.predict(x_for_hr)
      partial_hazard = [np.exp(lph) for lph in log_partial_hazard]

      treat_hr = []
      # ratio with treated and untreated
      for g in range(300):
        treat_hr.append([partial_hazard[g+302]/partial_hazard[g+2],'G' + str(g+1)])
      treat_hr.sort()

      # evaluation
      ev = EvalSurv(surv, durations_test, events_test, censor_surv='km')
      time_grid = np.linspace(durations_test.min(), durations_test.max(), 100)
      score = ev.integrated_brier_score(time_grid)

      if k == 0:
        brier_scores.append([score, i, j, lrfinder.get_best_lr(), treat_hr[:10]])
      else:
        brier_scores.append([score, i, j, k, treat_hr[:10]])

1 / 12
CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 10.7 µs
0:	[0s / 0s],		train_loss: 5.5708,	val_loss: 4.2832
1:	[0s / 0s],		train_loss: 5.5007,	val_loss: 4.2825
2:	[0s / 0s],		train_loss: 5.4480,	val_loss: 4.2815
3:	[0s / 0s],		train_loss: 5.3859,	val_loss: 4.2805
4:	[0s / 0s],		train_loss: 5.3485,	val_loss: 4.2793
5:	[0s / 0s],		train_loss: 5.2865,	val_loss: 4.2781
6:	[0s / 0s],		train_loss: 5.2485,	val_loss: 4.2767
7:	[0s / 0s],		train_loss: 5.2143,	val_loss: 4.2752
8:	[0s / 0s],		train_loss: 5.1714,	val_loss: 4.2736
9:	[0s / 0s],		train_loss: 5.1383,	val_loss: 4.2718
10:	[0s / 0s],		train_loss: 5.0919,	val_loss: 4.2699
11:	[0s / 0s],		train_loss: 5.0752,	val_loss: 4.2679
12:	[0s / 0s],		train_loss: 5.0217,	val_loss: 4.2659
13:	[0s / 0s],		train_loss: 4.9944,	val_loss: 4.2637
14:	[0s / 0s],		train_loss: 4.9890,	val_loss: 4.2615
15:	[0s / 1s],		train_loss: 4.9577,	val_loss: 4.2591
16:	[0s / 1s],		train_loss: 4.9173,	val_loss: 4.2567
17:	[0s / 1s],		train_loss: 4.8878

/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


2 / 12
CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 9.3 µs
0:	[0s / 0s],		train_loss: 5.6313,	val_loss: 4.2695
1:	[0s / 0s],		train_loss: 5.1959,	val_loss: 4.2622
2:	[0s / 0s],		train_loss: 4.9524,	val_loss: 4.2524
3:	[0s / 0s],		train_loss: 4.7523,	val_loss: 4.2417
4:	[0s / 0s],		train_loss: 4.5711,	val_loss: 4.2300
5:	[0s / 0s],		train_loss: 4.4134,	val_loss: 4.2183
6:	[0s / 0s],		train_loss: 4.2835,	val_loss: 4.2063
7:	[0s / 0s],		train_loss: 4.1448,	val_loss: 4.1940
8:	[0s / 0s],		train_loss: 4.0482,	val_loss: 4.1829
9:	[0s / 0s],		train_loss: 3.9455,	val_loss: 4.1734
10:	[0s / 0s],		train_loss: 3.8660,	val_loss: 4.1650
11:	[0s / 0s],		train_loss: 3.7733,	val_loss: 4.1572
12:	[0s / 0s],		train_loss: 3.7444,	val_loss: 4.1516
13:	[0s / 0s],		train_loss: 3.6796,	val_loss: 4.1437
14:	[0s / 0s],		train_loss: 3.8575,	val_loss: 4.1381
15:	[0s / 0s],		train_loss: 3.8222,	val_loss: 4.1347
16:	[0s / 1s],		train_loss: 3.7494,	val_loss: 4.1324
17:	[0s / 1s],		train_loss: 3.8153,

- brier_score가 가장 작은 것부터 정렬


In [14]:
brier_scores.sort()
selected_genes = []
for i in range(10):
  selected_genes.append(brier_scores[0][4][i][1])

In [15]:
gene_count= {}
for i in brier_scores:
  lst = []
  for j in i[4]:
    lst.append(j[1])
  print(lst)
  for k in lst:
    if k in gene_count.keys():
      gene_count[k] += 1
    else:
      gene_count[k] = 1
sorted(gene_count.items(), key=lambda x: x[1], reverse=True)

['G221', 'G255', 'G285', 'G243', 'G120', 'G215', 'G35', 'G67', 'G140', 'G125']
['G192', 'G284', 'G3', 'G243', 'G259', 'G217', 'G254', 'G127', 'G32', 'G182']
['G206', 'G14', 'G136', 'G21', 'G214', 'G86', 'G139', 'G164', 'G29', 'G203']
['G150', 'G60', 'G157', 'G43', 'G101', 'G14', 'G275', 'G137', 'G112', 'G243']
['G147', 'G236', 'G215', 'G174', 'G242', 'G119', 'G265', 'G149', 'G259', 'G180']
['G35', 'G118', 'G84', 'G186', 'G217', 'G193', 'G11', 'G258', 'G26', 'G34']
['G35', 'G274', 'G72', 'G149', 'G284', 'G34', 'G60', 'G77', 'G248', 'G43']
['G69', 'G153', 'G120', 'G284', 'G193', 'G205', 'G58', 'G136', 'G180', 'G37']
['G194', 'G165', 'G89', 'G185', 'G221', 'G183', 'G265', 'G120', 'G230', 'G60']
['G139', 'G236', 'G195', 'G252', 'G277', 'G242', 'G174', 'G230', 'G111', 'G35']
['G234', 'G175', 'G279', 'G127', 'G75', 'G55', 'G241', 'G262', 'G135', 'G275']
['G183', 'G256', 'G62', 'G75', 'G175', 'G220', 'G182', 'G118', 'G193', 'G35']


[('G35', 5),
 ('G243', 3),
 ('G120', 3),
 ('G284', 3),
 ('G60', 3),
 ('G193', 3),
 ('G221', 2),
 ('G215', 2),
 ('G259', 2),
 ('G217', 2),
 ('G127', 2),
 ('G182', 2),
 ('G14', 2),
 ('G136', 2),
 ('G139', 2),
 ('G43', 2),
 ('G275', 2),
 ('G236', 2),
 ('G174', 2),
 ('G242', 2),
 ('G265', 2),
 ('G149', 2),
 ('G180', 2),
 ('G118', 2),
 ('G34', 2),
 ('G183', 2),
 ('G230', 2),
 ('G175', 2),
 ('G75', 2),
 ('G255', 1),
 ('G285', 1),
 ('G67', 1),
 ('G140', 1),
 ('G125', 1),
 ('G192', 1),
 ('G3', 1),
 ('G254', 1),
 ('G32', 1),
 ('G206', 1),
 ('G21', 1),
 ('G214', 1),
 ('G86', 1),
 ('G164', 1),
 ('G29', 1),
 ('G203', 1),
 ('G150', 1),
 ('G157', 1),
 ('G101', 1),
 ('G137', 1),
 ('G112', 1),
 ('G147', 1),
 ('G119', 1),
 ('G84', 1),
 ('G186', 1),
 ('G11', 1),
 ('G258', 1),
 ('G26', 1),
 ('G274', 1),
 ('G72', 1),
 ('G77', 1),
 ('G248', 1),
 ('G69', 1),
 ('G153', 1),
 ('G205', 1),
 ('G58', 1),
 ('G37', 1),
 ('G194', 1),
 ('G165', 1),
 ('G89', 1),
 ('G185', 1),
 ('G195', 1),
 ('G252', 1),
 ('G277', 1),


- brier_score가 가장 좋은 케이스 출력

In [16]:
print("Brier Score :", brier_scores[0][0])
print("Hidden Layers :", brier_scores[0][1])
print("Number of Nodes :", brier_scores[0][2])
print("Learning Rate :", brier_scores[0][3])
print("Selection :", selected_genes)

Brier Score : 0.0979749663307075
Hidden Layers : 2
Number of Nodes : 2000
Learning Rate : 0.001
Selection : ['G221', 'G255', 'G285', 'G243', 'G120', 'G215', 'G35', 'G67', 'G140', 'G125']


NAN 값을 기준으로 정렬이 끊어진 문제 발견

In [17]:
print("brier_score, hidden layer, number of nodes, learning rate")
for i in brier_scores:
  print(i[:-1])

brier_score, hidden layer, number of nodes, learning rate
[0.0979749663307075, 2, 2000, 0.001]
[0.09969601026343161, 2, 3000, 0.001]
[0.10017798924924226, 2, 2000, 0.0001]
[0.10088860570725415, 1, 2000, 0.002154434690031894]
[0.10198860345920831, 1, 2000, 0.001]
[0.10244476416907433, 1, 3000, 0.000849753435908648]
[0.10246278171000388, 1, 3000, 0.001]
[0.10253665262191089, 2, 2000, 0.0001]
[0.10333631141200421, 1, 3000, 0.0001]
[0.10576775530568079, 2, 3000, 0.0001]
[0.10749559966507849, 1, 2000, 0.0001]
[0.10809572611835254, 2, 3000, 0.0001]
